<H1> Разработка инструмента для прогнозирования популярности постов в социальных сетях с применением методов машинного обучения </H1>

Проект содержит следующие этапы:
* Сбор БД и визуализация данных
* Токенизация, лемматизация и стемминг данных
* Векторизация
* Классификация
* **Подбор гиперпараметров и демонстрация итоговых результатов**

Для корректной работы кода необходимо загрузить файлы "ITMO_2.csv", "ITMO_2_vectorized_tf_idf
.npz",  полученные в рамках предыдущего шага проекта.

In [ ]:
# общие библиотеки
import csv
import re
import pandas as pd

# nlp-библиотеки для токенизации
%pip install nltk
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

# для лемматизации
%pip install pymorphy2 > None
import pymorphy2

# для стемминга
from nltk.stem.snowball import RussianStemmer

In [ ]:
text = 'Тот самый старт учебного года в ИТМО 😍 Так мы отрывались на ITMO GO и ITMOTION. Было всё: лазерное шоу, выступления творческих команд из ИТМО, мощный сет от Dj FeniХХХ и даже слэм! Смотрим отчет от Мегабайт Медиа и мысленно возвращаемся в этот незабываемый день 🔥'

**Токенизация, лемматизация и векторизация**

In [ ]:
import nltk
from string import punctuation

def remove_emojis(text):
    # регулярное выражение для удаления большинства смайликов, которые встречаются в постах
    emoji_pattern = re.compile(r'[^\u0000-\uD7FF\uE000-\uFFFF]', flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def tokenize_text(text):
    # загрузка стоп-слов на русском языке из nltk
    stop_words = set(nltk.corpus.stopwords.words('russian'))

    # кастомные стоп-слова
    custom_stop_words = {'это', 'вместе', 'а', 'где', 'https', 'с', 'в', 'по', 'далее', 'так', 'быть', 'ru', 'сс', 'сс', '00', '—', '⬇️', '✅', '✔️', '❗️', '⌚️'}
    stop_words.update(custom_stop_words)

    # удаление смайликов, URL-адресов и символов переноса строки
    text = remove_emojis(text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\n', ' ', text)

    # токенизация
    tokens = word_tokenize(text.lower())

    # удаление пунктуации и стоп-слов
    punctuations = list(punctuation)
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuations]

    return ' '.join(filtered_tokens)

# вывод на экран токенизированного текста
tokenized_text = tokenize_text(text)
print(tokenized_text)

самый старт учебного года итмо отрывались itmo go itmotion всё лазерное шоу выступления творческих команд итмо мощный сет dj feniххх слэм смотрим отчет мегабайт медиа мысленно возвращаемся незабываемый день


In [ ]:
import pymorphy2

def lemmatize_text(tokenized_text):
    # записываем в morph лемматизатор
    morph = pymorphy2.MorphAnalyzer()

    # лемматизируем текст
    tokens = tokenized_text.split()
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)

    return lemmatized_text

# лемматизация текста
lemmatized_text = lemmatize_text(tokenized_text)
print(lemmatized_text)

самый старт учебный год итмый отрываться itmo go itmotion всё лазерный шоу выступление творческий команда итмый мощный сет dj feniххха слэм смотреть отчёт мегабайт медиа мысленно возвращаться незабываемый день


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# загрузка данных из файла npz
data = np.load('ITMO_2_vectorized_bag_of_words.npz', allow_pickle=True)
feature_names = data['feature_names']
vectorizer = CountVectorizer(vocabulary=feature_names)

# векторизация текста
X_new = vectorizer.transform([lemmatized_text])
print(X_new.toarray())

[[0 0 0 ... 0 0 0]]


**Непосредственно обучение и классификация**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузка данных из файла .npz
data_npz = np.load('ITMO_2_vectorized_bag_of_words.npz', allow_pickle=True)

# Извлечение переменной 'data' из файла .npz
X_data = data_npz['X']

# Загрузка целевой переменной из файла ITMO_2.csv
y_data = pd.read_csv('ITMO_2.csv')['popular']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(
        penalty = 'l2',
        C = 0.2,
        class_weight = 'balanced',
        random_state = 42,
        max_iter = 100,
        n_jobs = -1
)

clf_lr = clf_lr.fit(X_train, y_train)
y_predict_lr = clf_lr.predict(X_test)

In [ ]:
# классификация нового текста
y_pred = clf_lr.predict(X_new)
print(f"Предсказанный класс: {y_pred}")

Предсказанный класс: [1]
